# Analyzing Thanksgiving Dinner

In this project, we'll be analyzing data on Thanksgiving dinner in the US. The dataset came from [FiveThirtyEight](fivethirtyeight.com), and can be found [here](https://github.com/fivethirtyeight/data/tree/master/thanksgiving-2015).

The dataset is stored in the `thanksgiving.csv` file. It contains 1058 responses to an online survey about what Americans eat for Thanksgiving dinner. Each survey respondent was asked questions about what they typically eat for Thanksgiving, along with some demographic questions, like their gender, income, and location. This dataset will allow us to discover regional and income-based patterns in what Americans eat for Thanksgiving dinner.

The dataset has 65 columns, and 1058 rows. Most of the column names are questions, and most of the column values are string responses to the questions. Most of the columns are categorical, as a survey respondent had to select one of a few options. For example, one of the first column names is What is typically the main dish at your Thanksgiving dinner?. The potential responses are:

- Turkey
- Other (please specify)
- Ham/Pork
- Tofurkey
- Chicken
- Roast beef
- I don't know
- Turducken

Most of the columns follow the same question/response format as the above. There are also quite a few NaN values in the columns, which occurred when a survey respondent didn't fill out a question because they didn't want to, or it didn't apply to them.

We won't enumerate every single column now, but here are descriptions of some of the most important:

- `RespondentID` -- a unique ID of the respondent to the survey.
- `Do you celebrate Thanksgiving?` -- a Yes/No reponse to the question.
- `How would you describe where you live?` -- responses are Suburban, Urban, and Rural.
- `Age` -- resposes are one of several categories, such as 18-29, and 30-44.
- `How much total combined money did all members of your HOUSEHOLD earn last year?` -- one of several categories, such as 75.000 USD to 99.999 USD.

In this project, we'll explore the data, and try to find interesting patterns. Our first step is to read in and display the data.

## Read and Display The Data

In [1]:
import pandas as pd
data = pd.read_csv(r"C:\Users\USER\Downloads\The Ultimate Plan\Jupyter Notebook\Datasets\thanksgiving.csv", encoding="Latin-1")
data[:3]

,RespondentID,Do you celebrate Thanksgiving?,What is typically the main dish at your Thanksgiving dinner?,What is typically the main dish at your Thanksgiving dinner? - Other (please specify),How is the main dish typically cooked?,How is the main dish typically cooked? - Other (please specify),What kind of stuffing/dressing do you typically have?,What kind of stuffing/dressing do you typically have? - Other (please specify),What type of cranberry saucedo you typically have?,What type of cranberry saucedo you typically have? - Other (please specify),...,Have you ever tried to meet up with hometown friends on Thanksgiving night?,"Have you ever attended a ""Friendsgiving?""",Will you shop any Black Friday sales on Thanksgiving Day?,Do you work in retail?,Will you employer make you work on Black Friday?,How would you describe where you live?,Age,What is your gender?,How much total combined money did all members of your HOUSEHOLD earn last year?,US Region
0,4337954960,Yes,Turkey,NaN,Baked,NaN,Bread-based,NaN,None,NaN,...,Yes,No,No,No,NaN,Suburban,18 - 29,Male,"$75,000 to $99,999",Middle Atlantic
1,4337951949,Yes,Turkey,NaN,Baked,NaN,Bread-based,NaN,Other (please specify),Homemade cranberry gelatin ring,...,No,No,Yes,No,NaN,Rural,18 - 29,Female,"$50,000 to $74,999",East South Central
2,4337935621,Yes,Turkey,NaN,Roasted,NaN,Rice-based,NaN,Homemade,NaN,...,Yes,Yes,Yes,No,NaN,Suburban,18 - 29,Male,"$0 to $9,999",Mountain


## Filtering Out Rows From A DataFrame
Because we want to understand what people ate for Thanksgiving, we'll remove any responses from people who don't celebrate it. The column Do you celebrate Thanksgiving? contains this information. We only want to keep data for people who answered Yes to this questions.

In [2]:
data.columns[:5]

Index(['RespondentID', 'Do you celebrate Thanksgiving?',
       'What is typically the main dish at your Thanksgiving dinner?',
       'What is typically the main dish at your Thanksgiving dinner? - Other (please specify)',
       'How is the main dish typically cooked?'],
      dtype='object')

In [3]:
data["Do you celebrate Thanksgiving?"].value_counts()

Yes    980
No      78
Name: Do you celebrate Thanksgiving?, dtype: int64

In [4]:
data = data[data["Do you celebrate Thanksgiving?"] == "Yes"]
data.shape

(980, 65)

## Using `Value_counts` To Explore Main Dishes
Let's explore what main dishes people tend to eat during Thanksgiving dinner. We can again use the value_counts method to help us with this.

In [5]:
data["What is typically the main dish at your Thanksgiving dinner?"].value_counts()

Turkey                    859
Other (please specify)     35
Ham/Pork                   29
Tofurkey                   20
Chicken                    12
Roast beef                 11
I don't know                5
Turducken                   3
Name: What is typically the main dish at your Thanksgiving dinner?, dtype: int64

In [6]:
tofurkey_data = data[data["What is typically the main dish at your Thanksgiving dinner?"] == "Tofurkey"]
tofurkey_data["Do you typically have gravy?"]

4      Yes
33     Yes
69      No
72      No
77     Yes
145    Yes
175    Yes
218     No
243    Yes
275     No
393    Yes
399    Yes
571    Yes
594    Yes
628     No
774     No
820     No
837    Yes
860     No
953    Yes
Name: Do you typically have gravy?, dtype: object

## Figuring Out What Pies People Eat
Now that we've looked into the main dishes, let's explore the dessert dishes. Specifically, we'll look at how many people eat Apple, Pecan, or Pumpkin pie during Thanksgiving dinner. This data is encoded in the following three columns:

- Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Apple
- Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pumpkin
- Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pecan

In all three columns, the value is either the name of the pie if the person eats it for Thanksgiving dinner, or null otherwise.

We can find out how many people eat one of these three pies for Thanksgiving dinner by figuring out for how many people all three columns are null.

You may recall from an earlier mission that the `pandas.isnull()` function will return a Boolean Series indicating whether or not each value in a specified DataFrame or Series is null.

We can also use the `&` operator to combine two Boolean Series into a single one. If both Series contain True in a position, the result will be True. Otherwise, the result will be False.

If we use the `pandas.isnull()` function to check where all three columns are null, then use the & operator to join all of the Series, we'll end up with a single Boolean Series. Where that Series contains False, the person ate at least one of the types of pies for Thanksgiving dinner. Where it contains True, they ate none of the types of pies.

In [7]:
apple_isnull = pd.isnull(data["Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Apple"])
pumpkin_isnull = pd.isnull(data["Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pumpkin"])
pecan_isnull = pd.isnull(data["Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pecan"])

In [8]:
ate_pies = apple_isnull & pumpkin_isnull & pecan_isnull
ate_pies.value_counts()

False    876
True     104
dtype: int64

## Converting Age To Numeric
Let's analyze the Age column in more depth. In order to analyze the Age column, we'll first need to convert it to numeric values. This will make it simple to figure out things like the average age of survey respondents. The Age column contains values that fall into one of a few categories:

- 18 - 29
- 30 - 44
- 45 - 59
- 60+
- null

Because we're missing the exact age value, we won't be able to extract an exact integer value, and we'll instead have to extract the first age value in the strings given.

We can do this by splitting each value on the space character (), then taking the first item in the resulting list. We'll also have to replace the + character to account for 60+, which follows a different format than the rest.

In [9]:
data["Age"].value_counts()

45 - 59    269
60+        258
30 - 44    235
18 - 29    185
Name: Age, dtype: int64

In [10]:
def extract_age(age_str):
    if pd.isnull(age_str):
        return None
    age_str = age_str.split(" ")[0]
    age_str = age_str.replace("+", "")
    return int(age_str)

data["int_age"] = data["Age"].apply(extract_age)
data["int_age"].describe()

count    947.000000
mean      40.089757
std       15.352014
min       18.000000
25%       30.000000
50%       45.000000
75%       60.000000
max       60.000000
Name: int_age, dtype: float64

### Findings
From 947 respondents, the average age is 40 years old, with up to 50% of the respondent.
Although we only have a rough approximation of age, and it skews downward because we took the first value in each string (the lower bound), we can see that that age groups of respondents are fairly evenly distributed.

## Converting Income To Numeric
The How much total combined money did all members of your HOUSEHOLD earn last year? column is very similar to the Age column. It contains categories, but can be converted to numerical values. Here are the unique values in the column:

- Prefer not to answer
- 0 to 9,999 USD
- 10,000 to 24,999 USD
- 25,000 to 49,999 USD
- 50,000 to 74,999 USD
- 75,000 to 99,999 USD
- 100,000 to 124,999 USD
- 125,000 to 149,999 USD
- 150,000 to 174,999 USD
- 175,000 to 199,999 USD
- 200,000 USD and up
- null

We can convert these values to numeric by again splitting on the space character (). We'll then have to account for the string Prefer. Finally, we'll be able to replace the dollar sign character $ and the comma ,, and return the result.

In [11]:
data["How much total combined money did all members of your HOUSEHOLD earn last year?"].value_counts()

$25,000 to $49,999      166
$50,000 to $74,999      127
$75,000 to $99,999      127
Prefer not to answer    118
$100,000 to $124,999    109
$200,000 and up          76
$10,000 to $24,999       60
$0 to $9,999             52
$125,000 to $149,999     48
$150,000 to $174,999     38
$175,000 to $199,999     26
Name: How much total combined money did all members of your HOUSEHOLD earn last year?, dtype: int64

In [12]:
def extract_income(income_str):
    if pd.isnull(income_str):
        return None
    income_str = income_str.split(" ")[0]
    if income_str == "Prefer":
        return None
    income_str = income_str.replace(",", "")
    income_str = income_str.replace("$", "")
    return int(income_str)

data["int_income"] = data["How much total combined money did all members of your HOUSEHOLD earn last year?"].apply(extract_income)
data["int_income"].describe()

count       829.000000
mean      75965.018094
std       59068.636748
min           0.000000
25%       25000.000000
50%       75000.000000
75%      100000.000000
max      200000.000000
Name: int_income, dtype: float64

## Findings
Although we only have a rough approximation of income, and it skews downward because we took the first value in each string (the lower bound), the average income seems to be fairly high, although there is also a large standard deviation.

In [13]:
data["int_income"].value_counts()

25000.0     166
75000.0     127
50000.0     127
100000.0    109
200000.0     76
10000.0      60
0.0          52
125000.0     48
150000.0     38
175000.0     26
Name: int_income, dtype: int64

## Correlating Travel Distance And Income
We can now see how the distance someone travels for Thanksgiving dinner relates to their income level. It's safe to hypothesize that people earning less money could be younger, and would travel to their parent's houses for Thanksgiving. People earning more are more likely to have Thanksgiving at their house as a result.

We can test this by filtering data based on int_income, and seeing what the values in the `How far will you travel for Thanksgiving?` column are.

In [14]:
less_than_fifteen = data[data["int_income"] < 150000.0]["How far will you travel for Thanksgiving?"]
less_than_fifteen.value_counts()

Thanksgiving is happening at my home--I won't travel at all                         281
Thanksgiving is local--it will take place in the town I live in                     203
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    150
Thanksgiving is out of town and far away--I have to drive several hours or fly       55
Name: How far will you travel for Thanksgiving?, dtype: int64

In [15]:
fifteen = data[data["int_income"] == 150000.0]["How far will you travel for Thanksgiving?"]
fifteen.value_counts()

Thanksgiving is happening at my home--I won't travel at all                         17
Thanksgiving is local--it will take place in the town I live in                      9
Thanksgiving is out of town but not too far--it's a drive of a few hours or less     9
Thanksgiving is out of town and far away--I have to drive several hours or fly       3
Name: How far will you travel for Thanksgiving?, dtype: int64

## Linking Friendship And Age
There are two columns which directly pertain to friendship, `Have you ever tried to meet up with hometown friends on Thanksgiving night?`, and `Have you ever attended a "Friendsgiving?"`. In the US, a "Friendsgiving" is when instead of traveling home for the holiday, you celebrate it with friends who live in your area. Both questions seem skewed towards younger people. Let's see if this hypothesis holds up.

In order to see the average ages of people who have done both, we can use a pivot table. As you may recall from an earlier mission, we can generate a pivot table with the [pandas.DataFrame.pivot_table()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pivot_table.html) method. By calling this method on data, and passing in the right keyword arguments, we can generate a table showing the average ages of people who answered Yes to both questions, answered Yes to one question, and so on.

In [16]:
data.pivot_table(index="Have you ever tried to meet up with hometown friends on Thanksgiving night?", columns='Have you ever attended a "Friendsgiving?"', values="int_age")

"Have you ever attended a ""Friendsgiving?""",No,Yes
Have you ever tried to meet up with hometown friends on Thanksgiving night?,,
No,42.283702,37.010526
Yes,41.475410,33.976744


## Here are some potential next steps
- Figure out the most common dessert people eat.
- Figure out the most common complete meal people eat.
- Identify how many people work on Thanksgiving.
- Find regional patterns in the dinner menus.
- Find age, gender, and income based patterns in dinner menus.